In [ ]:
#Shikha Pandey (2462279)

import numpy as np

class CustomDecisionTree:
    def __init__(self, max_depth=None):
        """
        Initialize the decision tree object.

        Attributes:
        max_depth (int): Maximum depth of the tree. Stops growing tree beyond this.
        tree (dict): Stores the trained tree after calling fit.
        """
        self.max_depth = max_depth  # Maximum depth allowed for tree (None = unlimited)
        self.tree = None            # Placeholder for tree structure (filled in fit method)

    def fit(self, X, y):
        """
        Train the decision tree on the dataset.

        Parameters:
        X (np.array): Feature matrix (n_samples x n_features)
        y (np.array): Target labels (n_samples,)

        Action:
        Builds a tree using recursive _build_tree method and stores in self.tree.
        """
        # Start recursive building from root (depth=0)
        self.tree = self._build_tree(X, y)

    def _build_tree(self, X, y, depth=0):
        """
        Recursively constructs the decision tree.

        Parameters:
        X (np.array): Features for the current node
        y (np.array): Labels for the current node
        depth (int): Current depth of recursion

        Returns:
        dict: Tree node represented as a dictionary. Leaf nodes have 'class'.
        """

        # Number of samples and features at current node
        num_samples, num_features = X.shape

        # Unique classes present at current node
        unique_classes = np.unique(y)

        # ---------------- Stopping conditions ----------------
        # 1. Node is "pure" (all samples belong to one class)
        if len(unique_classes) == 1:
            return {'class': unique_classes[0]}  # Leaf node with that class

        # 2. Max depth reached or no samples
        if num_samples == 0 or (self.max_depth is not None and depth >= self.max_depth):
            # Leaf node with majority class
            return {'class': np.bincount(y).argmax()}

        # ---------------- Find the best split ----------------
        best_info_gain = -float('inf')  # Start with worst possible info gain
        best_split = None               # Will store feature, threshold, and masks

        # Loop over all features to find the one that gives best split
        for feature_idx in range(num_features):
            # Consider all unique values in this feature as potential split thresholds
            thresholds = np.unique(X[:, feature_idx])
            for threshold in thresholds:
                # Create boolean masks to split data
                left_mask = X[:, feature_idx] <= threshold   # Samples going left
                right_mask = ~left_mask                       # Samples going right

                # Labels for left and right splits
                left_y = y[left_mask]
                right_y = y[right_mask]

                # Compute information gain from this split
                info_gain = self._information_gain(y, left_y, right_y)

                # Update best split if IG is higher than previous best
                if info_gain > best_info_gain:
                    best_info_gain = info_gain
                    best_split = {
                        'feature_idx': feature_idx,  # Feature used for split
                        'threshold': threshold,      # Threshold value for split
                        'left_mask': left_mask,      # Boolean mask for left child
                        'right_mask': right_mask     # Boolean mask for right child
                    }

        # If no split improves info gain, create leaf with majority class
        if best_split is None:
            return {'class': np.bincount(y).argmax()}

        # ---------------- Recursion: build child nodes ----------------
        # Recursively build left subtree
        left_tree = self._build_tree(X[best_split['left_mask']], y[best_split['left_mask']], depth + 1)

        # Recursively build right subtree
        right_tree = self._build_tree(X[best_split['right_mask']], y[best_split['right_mask']], depth + 1)

        # Return node as dictionary
        return {
            'feature_idx': best_split['feature_idx'],  # Feature to split on
            'threshold': best_split['threshold'],      # Threshold to compare
            'left_tree': left_tree,                    # Left child subtree
            'right_tree': right_tree                   # Right child subtree
        }

    def _information_gain(self, parent, left, right):
        """
        Calculate information gain from a split.

        Information Gain = Entropy(parent) - Weighted average of children entropies

        Parameters:
        parent (np.array): Labels of current node before split
        left (np.array): Labels of left child
        right (np.array): Labels of right child

        Returns:
        float: Information gain value
        """
        parent_entropy = self._entropy(parent)    # Entropy before split
        left_entropy = self._entropy(left)        # Entropy of left child
        right_entropy = self._entropy(right)      # Entropy of right child

        # Weighted average entropy of children
        weighted_avg_entropy = (len(left)/len(parent))*left_entropy + (len(right)/len(parent))*right_entropy

        # Information gain
        return parent_entropy - weighted_avg_entropy

    def _entropy(self, y):
        """
        Compute entropy of label array y.

        Entropy measures impurity of a set of labels:
        H(y) = - sum(p_i * log2(p_i)) over all classes

        Parameters:
        y (np.array): Labels of node

        Returns:
        float: Entropy value
        """
        if len(y) == 0:
            return 0  # Empty node has 0 entropy

        # Count occurrences of each class
        class_counts = np.bincount(y)
        class_probs = class_counts / len(y)  # Convert to probabilities

        # Entropy formula
        return -np.sum(class_probs * np.log2(class_probs + 1e-9))  # Add epsilon to avoid log(0)

    def predict(self, X):
        """
        Predict class labels for samples in X.

        Parameters:
        X (np.array): Feature matrix (n_samples x n_features)

        Returns:
        list: Predicted class labels
        """
        # For each sample, traverse the tree recursively to get prediction
        return [self._predict_single(x, self.tree) for x in X]

    def _predict_single(self, x, tree):
        """
        Predict class label for a single sample by traversing the tree.

        Parameters:
        x (np.array): Single feature vector
        tree (dict): Current node/subtree in tree

        Returns:
        int: Predicted class label
        """
        # If current node is a leaf, return its class
        if 'class' in tree:
            return tree['class']

        # Decide to go left or right based on feature value
        feature_val = x[tree['feature_idx']]
        if feature_val <= tree['threshold']:
            return self._predict_single(x, tree['left_tree'])
        else:
            return self._predict_single(x, tree['right_tree'])


1. Classification models (Wine dataset)

In [4]:
from sklearn.datasets import load_wine
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import f1_score

# 1) Load data
data = load_wine()  # 178 samples, 13 numeric features, 3 classes [0,1,2][web:4][web:17]
X, y = data.data, data.target

# 2) Train–test split
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

# 3) Decision Tree classifier
dt_clf = DecisionTreeClassifier(random_state=42)
dt_clf.fit(X_train, y_train)
y_pred_dt = dt_clf.predict(X_test)
f1_dt = f1_score(y_test, y_pred_dt, average="macro")

# 4) Random Forest classifier (default)
rf_clf = RandomForestClassifier(random_state=42)
rf_clf.fit(X_train, y_train)
y_pred_rf = rf_clf.predict(X_test)
f1_rf = f1_score(y_test, y_pred_rf, average="macro")

print("F1 (DecisionTreeClassifier):", f1_dt)
print("F1 (RandomForestClassifier default):", f1_rf)


F1 (DecisionTreeClassifier): 0.9457411645054665
F1 (RandomForestClassifier default): 1.0


2. Hyperparameter tuning (RandomForestClassifier + GridSearchCV)

In [5]:
from sklearn.model_selection import GridSearchCV

param_grid = {
    "n_estimators": [50, 100, 200],
    "max_depth": [None, 5, 10, 20],
    "max_features": ["sqrt", "log2", None],
}

rf_base = RandomForestClassifier(random_state=42)

grid_search = GridSearchCV(
    estimator=rf_base,
    param_grid=param_grid,
    cv=5,
    scoring="f1_macro",
    n_jobs=-1
)

grid_search.fit(X_train, y_train)

print("Best params (RF classifier):", grid_search.best_params_)
best_rf_clf = grid_search.best_estimator_

y_pred_best = best_rf_clf.predict(X_test)
f1_best = f1_score(y_test, y_pred_best, average="macro")

print("F1 (RandomForestClassifier tuned):", f1_best)


Best params (RF classifier): {'max_depth': None, 'max_features': 'sqrt', 'n_estimators': 50}
F1 (RandomForestClassifier tuned): 1.0


3. Regression models + RandomizedSearchCV

In [6]:
import numpy as np
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import RandomizedSearchCV
from scipy.stats import randint, uniform

# Example: make a synthetic continuous target from labels
rng = np.random.RandomState(42)
y_reg = y + rng.normal(scale=0.1, size=y.shape)

X_train_r, X_test_r, y_train_r, y_test_r = train_test_split(
    X, y_reg, test_size=0.2, random_state=42
)

# 1) Decision Tree Regressor
dt_reg = DecisionTreeRegressor(random_state=42)
dt_reg.fit(X_train_r, y_train_r)
r2_dt = dt_reg.score(X_test_r, y_test_r)

# 2) Random Forest Regressor (default)
rf_reg = RandomForestRegressor(random_state=42)
rf_reg.fit(X_train_r, y_train_r)
r2_rf = rf_reg.score(X_test_r, y_test_r)

print("R2 (DecisionTreeRegressor):", r2_dt)
print("R2 (RandomForestRegressor default):", r2_rf)


R2 (DecisionTreeRegressor): 0.630380134288053
R2 (RandomForestRegressor default): 0.8781175141938282


In [7]:
# Distributions for random search
param_dist = {
    "n_estimators": randint(50, 300),
    "max_depth": [None, 5, 10, 20, 30],
    "min_samples_split": randint(2, 10),
    # you can also try "max_features": ["sqrt", "log2", 0.5]
}

rf_reg_base = RandomForestRegressor(random_state=42)

rand_search = RandomizedSearchCV(
    estimator=rf_reg_base,
    param_distributions=param_dist,
    n_iter=20,
    cv=5,
    scoring="r2",
    random_state=42,
    n_jobs=-1
)

rand_search.fit(X_train_r, y_train_r)

print("Best params (RF regressor):", rand_search.best_params_)
best_rf_reg = rand_search.best_estimator_

r2_best_rf = best_rf_reg.score(X_test_r, y_test_r)
print("R2 (RandomForestRegressor tuned):", r2_best_rf)


Best params (RF regressor): {'max_depth': 5, 'min_samples_split': 5, 'n_estimators': 138}
R2 (RandomForestRegressor tuned): 0.8836004814784975


Load and Split the Iris Datasets

In [8]:
# Necessary Imports
import numpy as np
import pandas as pd
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score
# Load the Iris dataset
data = load_iris()
X = data.data
y = data.target
# Split into training and test sets (80% training, 20% test)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

Train and Evaluate a Custom Decision Tree

In [9]:
# Train the custom decision tree
custom_tree = CustomDecisionTree(max_depth=3)
custom_tree.fit(X_train, y_train)
# Predict on the test set
y_pred_custom = custom_tree.predict(X_test)
# Calculate accuracy
accuracy_custom = accuracy_score(y_test, y_pred_custom)
print(f"Custom Decision Tree Accuracy: {accuracy_custom:.4f}")

Custom Decision Tree Accuracy: 1.0000


Train and Evaluate a Scikit Learn Decision Tree

In [10]:
# Train the Scikit-learn decision tree
sklearn_tree = DecisionTreeClassifier(max_depth=3, random_state=42)
sklearn_tree.fit(X_train, y_train)
# Predict on the test set
y_pred_sklearn = sklearn_tree.predict(X_test)
# Calculate accuracy
accuracy_sklearn = accuracy_score(y_test, y_pred_sklearn)
print(f"Scikit-learn Decision Tree Accuracy: {accuracy_sklearn:.4f}")

Scikit-learn Decision Tree Accuracy: 1.0000


Result Comparison

In [11]:
print(f"Accuracy Comparison:")
print(f"Custom Decision Tree: {accuracy_custom:.4f}")
print(f"Scikit-learn Decision Tree: {accuracy_sklearn:.4f}")

Accuracy Comparison:
Custom Decision Tree: 1.0000
Scikit-learn Decision Tree: 1.0000
